In [1]:
import datetime
from datetime import timedelta

import pandas as pd
import pyodbc
import sqlalchemy as db

import itertools
import investpy
import pandas_datareader.data as pdr


today = datetime.date.today()

In [ ]:
connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=xxx;DATABASE=xxx;UID=xxx;PWD=xxx')
cursor = connection.cursor()

In [ ]:
tabelka_id = pd.read_excel(
    r'....idbenchmark.xlsx')

In [ ]:
last_date = []
tabelka_id['Last date'] = 'nan'
for idek in tabelka_id['id']:
    cursor.execute(f'select top 1 date from Tam_bloomberg_data where securityid = {idek} order by date desc')
    for row in cursor:     
        tabelka_id['Last date'][tabelka_id['id'].isin([idek])] = row[0] + timedelta(days=1)
    connection.commit()
tabelka_id

In [5]:
def download_invest(index,today,last_date):
    search_result = investpy.search_quotes(text=index,products=['indices']
                                       , n_results=1)
    if index == 'MSCI Emerging Markets' or index == 'MSCI EM Net USD':
        search_result.country = 'world'
    last_date = str(last_date.strftime('%d/%m/%Y'))
    today = str(today.strftime('%d/%m/%Y'))
    df = investpy.get_index_historical_data(index=index,
                                        country=search_result.country,
                                        from_date=last_date,
                                        to_date=today,
                                       ).reset_index()
#     i = df['Date'],df['Close']
    return df['Date'],df['Close']

In [6]:
def download_stooq(index,today,last_date):

    df = pdr.DataReader(index,'stooq',last_date,today).reset_index()
#     s = df['Date'],df['Close']
    return df['Date'],df['Close']

In [7]:
def fred(index,last_date):
    df = pdr.DataReader(index,'fred',last_date).reset_index()
    return df['DATE'] ,df['GOLDAMGBD228NLBM']

In [8]:
def insertowanie(index,date,value):
    insert_value_price = [index,date,value]
    insert_query_price ='''insert into Tam_Bloomberg_data (securityid, Date, Value) 
    VALUES (?,?,?);'''


    cursor.execute(insert_query_price,insert_value_price)
    connection.commit()

In [9]:
insertki = []
for index,row in tabelka_id.iterrows():
    if row['Source'] == 'stooq':
        date, close = download_stooq(row['Ticker'],today,row['Last date'])
        insertki.append([row['id'], date, close])
    elif row['Source'] == 'invest':
        print(row['Ticker'])
        date, close = download_invest(row['Ticker'],today,row['Last date'])
        insertki.append([row['id'], date, close])
#     elif row['Source'] == 'fred':
#         date, close = fred(row['Ticker'],row['Last date'])
#         if close is str:
#             close = 0
#         else: 
#             continue
        insertki.append([row['id'], date, close])
    else:
        continue


MSCI EM Net USD
MSCI India Net USD
MSCI China Net USD
Investor MS
MSCI Emerging Markets
MSCI Turkey


In [10]:
insertki

[[176,
  0    2022-08-09
  1    2022-08-10
  2    2022-08-11
  3    2022-08-12
  4    2022-08-15
  5    2022-08-16
  6    2022-08-17
  7    2022-08-18
  8    2022-08-19
  9    2022-08-22
  10   2022-08-23
  11   2022-08-24
  12   2022-08-25
  13   2022-08-26
  14   2022-08-29
  15   2022-08-30
  16   2022-08-31
  Name: Date, dtype: datetime64[ns],
  0     504.08
  1     501.93
  2     510.60
  3     512.95
  4     512.05
  5     511.69
  6     512.41
  7     509.43
  8     505.38
  9     500.58
  10    499.42
  11    497.24
  12    506.56
  13    508.12
  14    500.63
  15    501.20
  16    501.91
  Name: Close, dtype: float64],
 [176,
  0    2022-08-09
  1    2022-08-10
  2    2022-08-11
  3    2022-08-12
  4    2022-08-15
  5    2022-08-16
  6    2022-08-17
  7    2022-08-18
  8    2022-08-19
  9    2022-08-22
  10   2022-08-23
  11   2022-08-24
  12   2022-08-25
  13   2022-08-26
  14   2022-08-29
  15   2022-08-30
  16   2022-08-31
  Name: Date, dtype: datetime64[ns],
  0     504.0

In [11]:
t=0
while t < len(insertki):
    for i,d,v in zip(itertools.repeat(insertki[t][0],len(insertki[t][1])),insertki[t][1],insertki[t][2]):
        insertowanie(i,d,v)
    t+=1

In [29]:
# last_date = tabelka_id['Last date'][4]
# download_invest('IMS',today,last_date)

RuntimeError: ERR#0045: index ims not found, check if it is correct.

In [13]:
tabelka_id

,id,Source,Ticker,Last date
0,176,invest,MSCI EM Net USD,2022-08-09 00:00:00
1,177,invest,MSCI India Net USD,2022-08-09 00:00:00
2,178,invest,MSCI China Net USD,2022-08-09 00:00:00
3,179,fred,GOLDAMGBD228NLBM,2022-08-11 00:00:00
4,186,invest,Investor MS,2022-08-11 00:00:00
5,181,invest,MSCI Emerging Markets,2022-08-09 00:00:00
6,189,stooq,^TBSP,2022-08-11 00:00:00
7,192,client,Robert,2022-07-30 00:00:00
8,175,client,MSCI Russia,2022-03-09 00:00:00
9,191,invest,MSCI Turkey,2022-08-09 00:00:00


In [14]:
# delete duplicated values

# WITH CTE(securityid, 
#     date ,
# 	duplicatecount)

# AS (SELECT securityid, 
#            date ,
#            ROW_NUMBER() OVER(PARTITION BY securityid, 
#                                           date
#            ORDER BY ID) AS DuplicateCount
#     FROM Tam_Bloomberg_data)
# DELETE FROM CTE
# WHERE DuplicateCount > 1